<a href="https://colab.research.google.com/github/shoushou0315/pointE/blob/main/PointE_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

論文  
https://arxiv.org/abs/2212.08751<br>
<br>
GitHub<br>
https://github.com/openai/point-e<br>
<br>
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/PointE_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 環境セットアップ

## GPU確認

In [1]:
!nvidia-smi

Tue Sep  9 14:58:26 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## GitHubからコード取得

In [2]:
%cd /content

!git clone https://github.com/openai/point-e.git

# using Commits on Dec 20, 2022
%cd /content/point-e
!git checkout fc8a607c08a3ea804cc82bf1ef8628f88a3a5d2f

/content
Cloning into 'point-e'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 57 (delta 4), reused 2 (delta 2), pack-reused 27 (from 1)
Receiving objects: 100% (57/57), 1.57 MiB | 9.90 MiB/s, done.
Resolving deltas: 100% (7/7), done.
/content/point-e
Note: switching to 'fc8a607c08a3ea804cc82bf1ef8628f88a3a5d2f'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at fc8a607 Merge pull request #5 from openai/hj/lice

## ライブラリのインストール

In [3]:
%cd /content/point-e

!pip install -e .

/content/point-e
Obtaining file:///content/point-e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-21pi0wup/clip_e70fccb4070445699eef41a1668bd4bd
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-21pi0wup/clip_e70fccb4070445699eef41a1668bd4bd
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.0 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=6b7b04bbdd41732f6f00541142a801f264cac6573d65edde01e0c34c8d6ac934
  Stored in directory: /tmp/pip-ephem-wheel-cache-e6_1fuzh/wheels/35/3e/df/3d24cbfb3b6a06f17a2bfd7d1138900d4365d9028aa8f6e92f
Successfully built clip
  Running setup.py develop for point-e


## ライブラリのインポート

In [4]:
%cd /content/point-e

from PIL import Image
from tqdm.auto import tqdm
import plotly.graph_objects as go

import torch
device = 'cuda' if torch.cuda.is_available() else "cpu"
print("using device is", device)

# for avoiding module 'skimage' has no attribute 'measure'
import skimage.measure
skimage.measure.label

from point_e.diffusion.configs import DIFFUSION_CONFIGS, diffusion_from_config
from point_e.diffusion.sampler import PointCloudSampler
from point_e.models.download import load_checkpoint
from point_e.models.configs import MODEL_CONFIGS, model_from_config
from point_e.util.plotting import plot_point_cloud
from point_e.util.point_cloud import PointCloud
from point_e.util.pc_to_mesh import marching_cubes_mesh

/content/point-e
using device is cuda


# Image to PointCloud

## 学習済みモデルのセットアップ

In [ ]:
print('creating base model...')
base_name = 'base40M' # @param ["base40M", "base300M", "base1B"]
base_model = model_from_config(MODEL_CONFIGS[base_name], device)
base_model.eval()
base_diffusion = diffusion_from_config(DIFFUSION_CONFIGS[base_name])

print('creating upsample model...')
upsampler_model = model_from_config(MODEL_CONFIGS['upsample'], device)
upsampler_model.eval()
upsampler_diffusion = diffusion_from_config(DIFFUSION_CONFIGS['upsample'])

print('downloading base checkpoint...')
base_model.load_state_dict(load_checkpoint(base_name, device))

print('downloading upsampler checkpoint...')
upsampler_model.load_state_dict(load_checkpoint('upsample', device))

In [ ]:
# build point cloud sampler
sampler = PointCloudSampler(
    device=device,
    models=[base_model, upsampler_model],
    diffusions=[base_diffusion, upsampler_diffusion],
    num_points=[1024, 4096 - 1024],
    aux_channels=['R', 'G', 'B'],
    guidance_scale=[3.0, 3.0],
)

## テスト画像のセットアップ

In [ ]:
# Load an image to condition on.
img = Image.open('/content/point-e/point_e/examples/example_data/cube_stack.jpg')
img

## Inference

In [ ]:
samples = None
for x in tqdm(sampler.sample_batch_progressive(batch_size=1, model_kwargs=dict(images=[img]))):
  samples = x

In [ ]:
pc = sampler.output_to_point_clouds(samples)[0]
fig = plot_point_cloud(pc, grid_size=3, fixed_bounds=((-0.75, -0.75, -0.75),(0.75, 0.75, 0.75)))

# Text to PointCloud

## 学習済みモデルのセットアップ

In [5]:
print('creating base model...')
base_name = 'base40M-textvec'
base_model = model_from_config(MODEL_CONFIGS[base_name], device)
base_model.eval()
base_diffusion = diffusion_from_config(DIFFUSION_CONFIGS[base_name])

print('creating upsample model...')
upsampler_model = model_from_config(MODEL_CONFIGS['upsample'], device)
upsampler_model.eval()
upsampler_diffusion = diffusion_from_config(DIFFUSION_CONFIGS['upsample'])

print('downloading base checkpoint...')
base_model.load_state_dict(load_checkpoint(base_name, device))

print('downloading upsampler checkpoint...')
upsampler_model.load_state_dict(load_checkpoint('upsample', device))

creating base model...


100%|███████████████████████████████████████| 890M/890M [00:10<00:00, 87.1MiB/s]


creating upsample model...
downloading base checkpoint...


  0%|          | 0.00/161M [00:00<?, ?iB/s]

downloading upsampler checkpoint...


  0%|          | 0.00/162M [00:00<?, ?iB/s]

<All keys matched successfully>

In [6]:
sampler = PointCloudSampler(
  device=device,
  models=[base_model, upsampler_model],
  diffusions=[base_diffusion, upsampler_diffusion],
  num_points=[1024, 4096 - 1024],
  aux_channels=['R', 'G', 'B'],
  guidance_scale=[3.0, 0.0],
  model_kwargs_key_filter=('texts', ''), # Do not condition the upsampler at all
)

In [ ]:
from google.colab import files
# Set a prompt to condition on.
prompt = 'A flat land of brown and green random distribution' # @param {type:"string"}

# Produce a sample from the model.
samples = None

for i in tqdm(range(100), desc="Generating point clouds"):
  for x in sampler.sample_batch_progressive(batch_size=1, model_kwargs=dict(texts=[prompt])):
    samples = x
  pc = sampler.output_to_point_clouds(samples)[0]
  #fig = plot_point_cloud(pc, grid_size=3, fixed_bounds=((-0.75, -0.75, -0.75),(0.75, 0.75, 0.75)))
  fig_plotly = go.Figure(
    data=[
      go.Scatter3d(
        x=pc.coords[:,0], y=pc.coords[:,1], z=pc.coords[:,2],
        mode='markers',
        marker=dict(
          size=2,
          color=['rgb({},{},{})'.format(r,g,b) for r,g,b in zip(pc.channels["R"], pc.channels["G"], pc.channels["B"])],
        )
      )
    ],
    layout=dict(
      scene=dict(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False)
      )
    ),
  )


  # Extract data from the plotly figure
  coords = pc.coords
  colors = pc.channels["R"], pc.channels["G"], pc.channels["B"]

  # Create a new PointCloud object to ensure the correct format for saving
  pc_to_save = PointCloud(coords=coords, channels={'R': colors[0], 'G': colors[1], 'B': colors[2]})

  # Write the point cloud to a PLY file
  with open(f'plotly_point_cloud_{i+100}.ply', 'wb') as f:
      pc_to_save.write_ply(f)
      files.download(f)

  print(f"Plotly point cloud saved to plotly_point_cloud_{i+100}.ply")

# Note: Plotting all 50 figures might be resource intensive.
# Consider only plotting a subset or saving them as files.
# fig_plotly.show(renderer="colab")

Generating point clouds:   0%|          | 0/100 [00:00<?, ?it/s]

Plotly point cloud saved to plotly_point_cloud_100.ply
Plotly point cloud saved to plotly_point_cloud_101.ply
Plotly point cloud saved to plotly_point_cloud_102.ply
Plotly point cloud saved to plotly_point_cloud_103.ply
Plotly point cloud saved to plotly_point_cloud_104.ply
Plotly point cloud saved to plotly_point_cloud_105.ply
Plotly point cloud saved to plotly_point_cloud_106.ply
Plotly point cloud saved to plotly_point_cloud_107.ply
Plotly point cloud saved to plotly_point_cloud_108.ply
Plotly point cloud saved to plotly_point_cloud_109.ply
Plotly point cloud saved to plotly_point_cloud_110.ply
Plotly point cloud saved to plotly_point_cloud_111.ply
Plotly point cloud saved to plotly_point_cloud_112.ply
Plotly point cloud saved to plotly_point_cloud_113.ply
Plotly point cloud saved to plotly_point_cloud_114.ply
Plotly point cloud saved to plotly_point_cloud_115.ply
Plotly point cloud saved to plotly_point_cloud_116.ply
Plotly point cloud saved to plotly_point_cloud_117.ply
Plotly poi

In [ ]:
import glob
from google.colab import files

# Get a list of all files starting with 'plot' and ending with '.ply'
ply_files = glob.glob('plot*.ply')

print(f"Found {len(ply_files)} .ply files to download:")

# Download each file
for file_path in ply_files:
    try:
        files.download(file_path)
        print(f"Downloaded: {file_path}")
    except Exception as e:
        print(f"Error downloading {file_path}: {e}")

# PointCloud to Mesh

## 学習済みモデルのセットアップ

In [ ]:
print('creating SDF model...')
name = 'sdf'
model = model_from_config(MODEL_CONFIGS[name], device)
model.eval()

print('loading SDF model...')
model.load_state_dict(load_checkpoint(name, device))

## PointCloudのセットアップ

In [ ]:
# Load a point cloud we want to convert into a mesh.
pc = PointCloud.load('/content/point-e/point_e/examples/example_data/pc_corgi.npz')

# Plot the point cloud as a sanity check.
fig = plot_point_cloud(pc, grid_size=2)

## Inference

In [ ]:
# Produce a mesh (with vertex colors)
mesh = marching_cubes_mesh(
  pc=pc,
  model=model,
  batch_size=4096,
  grid_size=32, # increase to 128 for resolution used in evals
  progress=True,
)

In [ ]:
# Write the mesh to a PLY file to import into some other program.
with open('mesh.ply', 'wb') as f:
  mesh.write_ply(f)